<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/notebooks/rl_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://raw.githubusercontent.com/Tinynja/Sarsa-phi-EB/master/notebooks/ALE_Framework_Tests.ipynb

%run ALE_Framework_Tests.ipynb

Cloning into 'Sarsa-phi-EB'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 326 (delta 106), reused 204 (delta 39), pack-reused 0
Receiving objects: 100% (326/326), 761.46 KiB | 12.69 MiB/s, done.
Resolving deltas: 100% (106/106), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.0.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
[SUPPORTED]          chopper_command ROMS/Chopper Command (1982).bin
[SU

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


Skipped background regeneration.
Skipped displaying stored backgrounds to reduce ouptuts.
Skipped manual test.


sarsa phi eb

In [ ]:
#######################################################################
# Copyright (C)                                                       #
# 2017-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from math import floor
from tqdm import tqdm


# all possible actions
ACTIONS = range(4)

# discount is always 1.0 in these experiments
DISCOUNT = 0.99

# use optimistic initial value, so it's ok to set epsilon to 0
EPSILON = 0.01

# maximum steps per episode
STEP_LIMIT = 5000


# get action at @position and @velocity based on epsilon greedy policy and @valueFunction  #########################    use our own get_action. modified it, may work as intended
def get_action(observation, valueFunction):
    if np.random.binomial(1, EPSILON) == 1:
        return np.random.choice(ACTIONS)
    values = []
    for action in ACTIONS:
        values.append(valueFunction.value(observation))  
    maxi = np.max(values)
    bestactions = np.where(values==maxi,1.0,0.0)
    for i in range(len(ACTIONS)):
      bestactions[i] = bestactions[i]*np.random.uniform()
    action = np.argmax(bestactions)
    return action



# replacing trace update rule
# @trace: old trace (will be modified)
# @activeTiles: current active tile indices
# @lam: lambda
# @return: new trace for convenience
def replacing_trace(trace, activeTiles, lam):
    active = (torch.arange(len(trace)).to(device)[None,...] == activeTiles.flatten()[...,None]).any(0)
    trace[active] = 1
    trace[~active] *= lam * DISCOUNT
    return trace



# wrapper class for Sarsa(lambda)
class Sarsa:
    # In this example I use the tiling software instead of implementing standard tiling by myself
    # One important thing is that tiling is only a map from (state, action) to a series of indices
    # It doesn't matter whether the indices have meaning, only if this map satisfy some property
    # View the following webpage for more information
    # http://incompleteideas.net/sutton/tiles/tiles3.html
    # @maxSize: the maximum # of indices
    #the hashing is a lfa?
    def __init__(self, step_size, lam, trace_update=replacing_trace, max_size=28672, initial_weights=0):
        self.max_size = max_size
        self.trace_update = trace_update
        self.lam = lam

        # divide step size equally to each tiling
        self.step_size = step_size/10

        # weight for each tile
        if initial_weights == 0:
          self.weights =torch.zeros(max_size).to(device) #max size is the number of features?
        else:
          self.weights = initial_weights.to(device)

        # trace for each tile
        self.trace = torch.zeros(max_size).to(device)



    # estimate the value of given state and action
    def value(self, observation):
        active_tiles = torch.nonzero(observation)
        return self.weights[active_tiles].sum()

    # learn with given state, action and target
    def learn(self, observation, target):
        active_tiles = torch.nonzero(observation)
        estimation = self.weights[active_tiles].sum()
        delta = target - estimation
        #print('estimation array: ' + str(self.weights[active_tiles]))
        print('estimation: ' + str(self.weights[active_tiles].sum()))
        if self.trace_update == replacing_trace:
            self.trace_update(self.trace, active_tiles, self.lam)
        else:
            raise Exception('Unexpected Trace Type')
        self.weights += self.step_size * delta * self.trace
        print('delta: ' + str(delta))
        print('weights: ' +  str(self.weights))


# play Mountain Car for one episode based on given method @evaluator
# @return: total steps in this episode
def play(evaluator, env):

    action = random.choice(ACTIONS)
    steps = 0
    while True:
        next_observation, reward, done, info = env.step(action)
        next_action = get_action(next_observation, evaluator)    #########################    use our own get_action  ??? modified it, may work as intented
        steps += 1
        target = reward + DISCOUNT * evaluator.value(next_observation)          ############# use our own value function ??? modified it, may work as intented
        evaluator.learn(observation, target)
        observation = next_observation
        action = next_action
        if done:
            break
        if steps >= STEP_LIMIT:
            print('Step Limit Exceeded!')
            break
    return steps

In [ ]:
class BaseAgent:
  """ The base agent class function.
  """
  def __init__(self, nb_features=28672):
    #nothing for now
    self.gamma = 1
    self.features = nb_features
    self.rhos = torch.ones(self.features).to(device) #stores the rho_i values


  def takeAction(self, t):
    phis = [[0,1,0],[0,1,0],[0,1,0],[1,0,1]]
    return phis[t]


  def updateRho_i(self, counts, t):
    M = self.features
    self.rhos = (counts+1.5)/(t+1)
    return 0


  def PHI_EB(self, evaluator, env, beta=0.05, t_end=200):
    t = 0
    M = self.features #number of features
    counts = torch.zeros(M).to(device)
    states = torch.zeros(t_end,M).to(device) #stores the previous phis for all timesteps

    action = 1 #starting the game for the agent on the first game
    old_phi = env._observe()
    print('starting iterations')
    print('rhos: ' + str(self.rhos))
    while t < t_end:
      print("Iteration #{}".format(t))
      #observe phi(s), reward
      phi, reward, done, info = env.step(action)
      print('--------------------------------------------------------------')
      print('took action: ', env.action_space[action])
      next_action = get_action(phi, evaluator)
      print('phi: ' + str(phi))
      
      #compute rho_t(phi) (feature visit-density)
      if t > 0:
        counts = (phi==states[0:t]).sum(0)
        # print(counts)
        self.rhos = (counts+0.5)/(t+1)
        print('rhos: ' + str(self.rhos))
        rho_t = torch.prod(self.rhos)
      else:
        rho_t = 0.5**M
      print('rho_t ' + str(rho_t))
      #update all rho_i with observed phi
      states[t] = phi
      self.updateRho_i(counts, t+1)
      print('min rho_i_t: ' + str(min(self.rhos)))
      
      #compute rho_t+1(phi)
      new_rho_t = 1
      for i in range(M):
        new_rho_t = new_rho_t*self.rhos[i]
      if new_rho_t <= 1e-323: #this is to avoid division by zero, might need to be tweaked
        new_rho_t = 1e-323
      print('new_rho_t ' + str(new_rho_t))

      #compute Nhat_t(s)
      Nhat_t = rho_t*(1-new_rho_t)/(new_rho_t-rho_t)
      print('Nhat_t: ',   Nhat_t)
      if Nhat_t <= 1e-323: #this is to avoid division by zero again, might need to be tweaked
        Nhat_t = torch.tensor([1e-323]).to(device)

      #compute R(s,a) (empirical reward)
      explorationBonus = beta/torch.sqrt(Nhat_t)
      if torch.isnan(explorationBonus) or explorationBonus >= 1e3:
        explorationBonus = 1e3

      reward = reward + explorationBonus
      print('reward: ',reward)

      print('state value: ' + str(evaluator.value(phi)))
      #pass phi(s) and reward to RL algo to update theta_t
      target = reward + self.gamma * evaluator.value(phi)          ############# use our own value function ??? modified it, may work as intented
      evaluator.learn(old_phi, target)

      if done:
        #break
        env.reset()
        action = 1
        old_phi = env._observe()
        print('episode ended on step ', t, 'starting a new one')
      else:
        old_phi = phi
        action = next_action

      t += 1

    return evaluator.weights


In [ ]:
from ale_py.roms import Breakout
env = EnvALE(Breakout, feature_type='Basic')
print(env.action_space)
alpha = 0.5
lam = 0.9
#we can upload previous weights as as tensor, or initialize at 0
previous_weights = 0


evaluator = Sarsa(alpha, lam, replacing_trace, 28672, previous_weights)
agent = BaseAgent()
env.reset(do_record=False)
weights = agent.PHI_EB(evaluator, env, beta=0.05, t_end=10000)


[<Action.NOOP: 0>, <Action.FIRE: 1>, <Action.RIGHT: 3>, <Action.LEFT: 4>]
starting iterations
rhos: tensor([1., 1., 1.,  ..., 1., 1., 1.])
Iteration #0
--------------------------------------------------------------
took action:  Action.FIRE
phi: tensor([ True, False, False,  ..., False, False, False])
rho_t 0.0
min rho_i_t: tensor(0.7500)
new_rho_t tensor(2.8026e-45)
Nhat_t:  tensor(0.)
reward:  1000.0
state value: tensor(0.)
estimation: tensor(0.)
delta: tensor(1000.)
weights: tensor([50.,  0.,  0.,  ...,  0.,  0.,  0.])
Iteration #1
--------------------------------------------------------------
took action:  Action.FIRE
phi: tensor([ True, False, False,  ..., False, False, False])
rhos: tensor([0.7500, 0.7500, 0.7500,  ..., 0.7500, 0.7500, 0.7500])
rho_t tensor(0.)
min rho_i_t: tensor(0.5000)
new_rho_t 1e-323
Nhat_t:  tensor(nan)
reward:  1000.0
state value: tensor(11500.)
estimation: tensor(11500.)
delta: tensor(1000.)
weights: tensor([100.,   0.,   0.,  ...,   0.,   0.,   0.])
Iter

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

df = pd.DataFrame(weights)
df.to_csv('weights.csv',index=False)

In [ ]:
torch.unique(weights)

tensor([-2.2507e+14, -2.0636e+14, -1.5559e+14, -1.4064e+14, -1.2728e+14,
        -1.2308e+14, -1.1768e+14, -8.7606e+13, -7.5274e+13, -7.5155e+13,
        -6.6346e+13, -6.0275e+13, -5.5657e+13, -4.2305e+13, -3.6145e+13,
        -2.4283e+13, -2.2198e+13,  0.0000e+00,  4.0416e+13,  5.3655e+13,
         6.4767e+13,  7.6218e+13,  8.0885e+13,  8.4170e+13,  8.9790e+13,
         9.7415e+13,  9.8442e+13,  9.9528e+13,  1.0906e+14,  1.1365e+14,
         1.1610e+14,  1.1933e+14,  1.1967e+14,  1.2303e+14,  1.2381e+14,
         1.2587e+14,  1.2837e+14,  1.2884e+14,  1.3777e+14,  1.3794e+14,
         1.4520e+14,  1.5968e+14,  1.7645e+14,  1.7650e+14,  1.7855e+14,
         1.8481e+14,  1.8539e+14,  1.9363e+14,  2.0072e+14,  2.0472e+14,
         2.1115e+14,  2.1236e+14,  2.1405e+14,  2.2247e+14,  2.3898e+14,
         2.4331e+14,  2.4374e+14,  2.4441e+14,  2.5321e+14,  2.5674e+14,
         2.6116e+14,  2.6192e+14,  2.6217e+14,  2.6347e+14,  2.7327e+14,
         2.9621e+14,  3.0445e+14,  3.0828e+14,  3.1

In [ ]:
##optimisation tests
# %%timeit
# t = 1
# M = 28000
# counts = np.zeros(M)
# rhos = np.ones(M)
# for i in range(M): #M is the number of features of phi
#   counts[i] += 1 #since we add phi to the seen states, all the counts are increased by one for t+1
#   rhos[i] = (counts[i]+0.5)/(t+1)

In [ ]:
# #best one
# %%timeit
# t = 1
# M = 28000
# counts = np.zeros(M)
# rhos = np.ones(M)
# rhos = (counts+1.5)/(t+1)

In [ ]:
# #compute rho_t(phi) (feature visit-density)
# %%timeit
# t = 0
# t_end = 30
# M = 10000
# phis = np.zeros((t_end,M))
# counts = np.zeros(M)
# states = np.zeros((t_end,M))
# rhos = np.ones(M)
# while t < t_end:
#   phi = phis[t]
#   if t > 0:
#     rho_t = 1
#     for i in range(M):
#       counts[i] = 0
#       for step in range(t):
#         if phi[i] == states[step,i]:
#           counts[i] += 1
#       rhos[i] = (counts[i]+0.5)/(t+1)
#       rho_t = rho_t*rhos[i]
#   else:
#     rho_t = 0.5**M

#   #updating rho (other method)
#   states[t] = phi
#   counts += 1
#   rhos = (counts+0.5)/(t+2)
  
#   t += 1

In [ ]:
# #compute rho_t(phi) (feature visit-density)
# %%timeit
# t = 0
# t_end = 5000
# M = 1000
# phis = np.zeros((t_end,M))
# counts = np.zeros(M)
# states = np.zeros((t_end,M))
# rhos = np.ones(M)
# while t < t_end:
#   phi = phis[t]
#   if t > 0:
#     rho_t = 1
#     counts = np.zeros(M)
#     for step in range(t):
#       counts[np.where(phi == states[step])] += 1
#     rhos = (counts+0.5)/(t+1)
#     rho_t = np.prod(rhos)
#   else:
#     rho_t = 0.5**M

#   #updating rho (other method)
#   states[t] = phi
#   counts += 1
#   rhos = (counts+0.5)/(t+2)
  
#   t += 1

In [ ]:
# #version 3 of optimisation
# %%timeit
# t = 0
# t_end = 5000
# M = 1000
# #phis = [[0,1,0],[0,1,0],[0,1,0],[1,1,0]]
# phis = np.zeros((t_end,M))
# counts = np.zeros(M)
# states = np.zeros((t_end,M))
# rhos = np.ones(M)
# while t < t_end:
#   phi = phis[t]
#   if t > 0:
#     rho_t = 1
#     counts = np.sum(np.where(phi == states[0:t],1,0),axis=0)
#     rhos = (counts+0.5)/(t+1)
#     rho_t = np.prod(rhos)
#   else:
#     rho_t = 0.5**M

#   #updating rho (other method)
#   states[t] = phi
#   counts += 1
#   rhos = (counts+0.5)/(t+2)
  
#   t += 1